In [44]:
import pandas as pd
import numpy as np 
import sys
from paths import Paths
paths = Paths()
sys.path.append(paths.scripts_root)
import util


#### Waveform Variable Range Table

|Var  |BI |AA |AB  |AC  |AD   |AE  |AF  |WT   |CA  |HT  |CB  |PA  |CC   |PB   |CD  |PC   |CE   |PD   |CF  |
|-----|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|Val 1| 20|10 | -25| 15 | -6  |3.6 |2.4 |1    |0.4 |0.6 |1   |0.1 |0.5  |0.2  |0.5 |0.3  |1    |1    | 0.5|
|Val 2|30 |5  |-10 |10  |-3.6 |-5  |3.6 |0.1  |1   |1.5 |1.5 |0.3 |0.8  |0.7  |1   |1.2  |1.5  |1.5  | 1.2|
|Fxn  |-|-|- |- |-  |- |- |CBxAF|- |- |- |- |HTxAE|CAxPD|- |-  |BIxCD|AAxPC|- |



In [2]:
WF_var_ranges = {
"BI": [20  , 30  , []],
"AA": [10  , 5   , []],
"AB": [-25 , -10 , []],
"AC": [15  , 10  , []],
"AD": [-6  , -3.6, []],
"AE": [3.6 , -5  , []],
"AF": [2.4 , 3.6 , []],
"WT": [1   , 0.1 , ["CB","AF"]],
"CA": [0.4 , 1   , []],
"HT": [0.6 , 1.5 , []],
"CB": [1   , 1.5 , []],
"PA": [0.1 , 0.3 , []],
"CC": [0.5 , 0.8 , ["HT","AE"]],
"PB": [0.2 , 0.7 , ["CA","PD"]],
"CD": [0.5 , 1   , []],
"PC": [0.3 , 1.2 , []],
"CE": [1   , 1.5 , ["BI","CD"]],
"PD": [1   , 1.5 , ["AA","PC"]],
"CF": [0.5 , 1.2 , []]
}

In [6]:
WF_var_header_fields = []
WF_var_dependents = {}
WF_var_independents = {}
WF_var_state = {}

num_independents = 0

def initialize_WF_vars():
    global num_independents
    for k,v in WF_var_ranges.items():
        WF_var_header_fields.append(k)
        if len(v[2]) > 0:
            WF_var_dependents[k] = v
        else:
            WF_var_independents[k] = v
            num_independents += 1
            
#         WF_var_values[k] = WF_var_ranges[k][0]
        WF_var_state[k] = 0
        #print(f"{k}: {WF_var_ranges[k][0]}")


initialize_WF_vars()
num_fields = len(WF_var_header_fields)

print("Dependent vars:", WF_var_dependents)
print(f"Independent vars ({num_independents}): {WF_var_independents}")

Dependent vars: {'WT': [1, 0.1, ['CB', 'AF']], 'CC': [0.5, 0.8, ['HT', 'AE']], 'PB': [0.2, 0.7, ['CA', 'PD']], 'CE': [1, 1.5, ['BI', 'CD']], 'PD': [1, 1.5, ['AA', 'PC']]}
Independent vars (14): {'BI': [20, 30, []], 'AA': [10, 5, []], 'AB': [-25, -10, []], 'AC': [15, 10, []], 'AD': [-6, -3.6, []], 'AE': [3.6, -5, []], 'AF': [2.4, 3.6, []], 'CA': [0.4, 1, []], 'HT': [0.6, 1.5, []], 'CB': [1, 1.5, []], 'PA': [0.1, 0.3, []], 'CD': [0.5, 1, []], 'PC': [0.3, 1.2, []], 'CF': [0.5, 1.2, []]}


In [7]:
# '{0:b}'.format(42)

def toBinary(n, l):
    return ''.join(str(1 & int(n) >> i) for i in range(l)[::-1])

state = toBinary(19, 20)
print(state)

00000000000000010011


In [8]:
'''
    Calculates the state and values for each waveform parameter
    
    Before calling, these must already be initialized:
        WF_var_dependents  : dict of list of ranges for dependent variables
        WF_var_independents: dict of list of ranges for independent variables
        WF_var_state: dict of states (0 or 1) for each variable
    
    Function will fill out:
        WF_var_state: dict of states (0 or 1) for each variable
        WF_var_values: dict of values for each variable
        
    Returns:
        ANSI binary string of states 
    
'''
def calc_vals(iter, debug=False):
    if debug: 
        print("States Before...")
        for idx,(k,v) in enumerate(WF_var_state.items()):
            var_type = "d" if k in WF_var_dependents else "i"
            print(f"[{idx:02}] ({var_type}) {k}: {v}")

    if debug: print(f"Dependent States: {toBinary(iter, len(WF_var_independents))}\n")

    '''
        Calc the independent variables.
        
            'BI': [20, 30, []]
    '''
    for idx,(k,v) in enumerate(WF_var_independents.items()):
        state = 1 & iter >> idx
        WF_var_state[k] = state
        WF_var_values[k] = v[state]

    ''' 
        Calc the dependent variables.

            'WT': [1, 0.1, ['CB', 'AF']]
    '''
    for idx,(k,v) in enumerate(WF_var_dependents.items()):
        if debug: print(f"dependent var {k} is {v[2][0]} xor {v[2][1]} -> {WF_var_state[v[2][0]]} xor {WF_var_state[v[2][1]]} = {WF_var_state[v[2][0]] ^ WF_var_state[v[2][1]]}")
        state = WF_var_state[v[2][0]] ^ WF_var_state[v[2][1]]
        WF_var_state[k] = state
        WF_var_values[k] = v[state]

    if debug: print("States After...")
    state_lst = list(''.join('-' for i in range(num_fields*2)[::-1]))
    for idx,(k,v) in enumerate(WF_var_state.items()):
        if k in WF_var_independents:
            ansi_code = util.BLUE
            var_type = "i"
        else:
            ansi_code = util.RED
            var_type = "d"
        if debug: print(f"[{idx:02}] ({var_type}) {k}: {v}")
        state_lst[idx*2] = ansi_code
        state_lst[(idx*2)+1] = str(v)
        state_str = ''.join(state_lst)

    if debug: print("Values After...")
    for idx,(k,v) in enumerate(WF_var_values.items()):
        if k in WF_var_dependents:
            var_type = "d"
            vals = WF_var_dependents[k][:2]
        else:
            var_type = "i"
            vals = WF_var_independents[k][:2]
        if debug: print(f"[{idx:02}] ({var_type}) {k}: {v} (state: {WF_var_state[k]}, vals{vals})")

    return state_str



In [12]:
WF_var_values = {}

iter = 220207
state_str = calc_vals(iter, True)
print(state_str+util.RESET)
print(WF_var_values)

States Before...
[00] (i) BI: 0
[01] (i) AA: 0
[02] (i) AB: 1
[03] (i) AC: 0
[04] (i) AD: 0
[05] (i) AE: 0
[06] (i) AF: 0
[07] (d) WT: 1
[08] (i) CA: 0
[09] (i) HT: 0
[10] (i) CB: 1
[11] (i) PA: 1
[12] (d) CC: 0
[13] (d) PB: 0
[14] (i) CD: 0
[15] (i) PC: 1
[16] (d) CE: 0
[17] (d) PD: 1
[18] (i) CF: 0
Dependent States: 01110000101111

dependent var WT is CB xor AF -> 0 xor 0 = 0
dependent var CC is HT xor AE -> 0 xor 1 = 1
dependent var PB is CA xor PD -> 0 xor 1 = 1
dependent var CE is BI xor CD -> 1 xor 1 = 0
dependent var PD is AA xor PC -> 1 xor 1 = 0
States After...
[00] (i) BI: 1
[01] (i) AA: 1
[02] (i) AB: 1
[03] (i) AC: 1
[04] (i) AD: 0
[05] (i) AE: 1
[06] (i) AF: 0
[07] (d) WT: 0
[08] (i) CA: 0
[09] (i) HT: 0
[10] (i) CB: 0
[11] (i) PA: 1
[12] (d) CC: 1
[13] (d) PB: 1
[14] (i) CD: 1
[15] (i) PC: 1
[16] (d) CE: 0
[17] (d) PD: 0
[18] (i) CF: 0
Values After...
[00] (i) BI: 30 (state: 1, vals[20, 30])
[01] (i) AA: 5 (state: 1, vals[10, 5])
[02] (i) AB: -10 (state: 1, vals[-25, -10]

In [10]:
cnt = pow(2,(len(WF_var_independents)))

s = set()
for iter in range(cnt):
    state_str = calc_vals(iter, False)
    #print(state_str+util.RESET)
    s.add(state_str)
print(f"cnt: {cnt}, set len: {len(s)}")

cnt: 16384, set len: 16384


### Oce Waveform Format

##### First 3 rows

|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|
|-----|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|LIB |2       |&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;
|SERI|2D5FV32H|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;
|#   |row     |column|BI    |AA    |AB    |AC    |AD    |AE    |AF    |WT    |CA    |HT    |CB    |PA    |CC    |PB    |CD    |PC    |CE    |PD    |CF    

#, [row, column, BI,    AA,    AB,    AC,    AD,    AE,    AF,    WT,    CA,    HT,    CB,    PA,    CC,    PB,    CD,    PC,    CE,    PD,    CF]

In [97]:
'''
    Create dataframe, then add serial number row
'''
def initialize_waveform_df(rows_per_head, nozzles_per_row, lib_cnt, col_cnt ):
    num_rows = 1 + (((rows_per_head*nozzles_per_row)+2)*lib_cnt)
    
    # pre-allocate the rows of the dataframe...
    df = pd.DataFrame(index=np.arange(0, num_rows), columns=[str(x) for x in range(col_cnt)])
    return df

'''
    Add serial number row to waveform dataframe
'''
def add_serial_data(df, serial_num_str):
    seri_data = ["SERI", serial_num_str]
    df.loc[0] = seri_data + ["" for x in range(len(df.columns)-len(seri_data))]
    
def add_nozzle_data(df, start_loc, lib_num, rows_per_head, nozzles_per_row, wf_data):
    waveform_cols = ['BI', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'WT', 'CA', 'HT', 'CB', 'PA', 'CC', 'PB', 'CD', 'PC', 'CE', 'PD', 'CF']
    
    start_index = 1 + (rows_per_head*nozzles_per_row*start_loc)

    lib_data = ["LIB", str(lib_num)]
    df.loc[start_index + 0] = lib_data + ["" for x in range(len(df.columns)-len(lib_data))]
    df.loc[start_index + 1] = ['#', 'row', 'column'] + waveform_cols

    for nozzle_row in range( 0, rows_per_head):
        for nozzle_col in range( 0, nozzles_per_row):
            nozzle_row_data = ["NOZ"]
            nozzle_row_data.append(nozzle_row)
            nozzle_row_data.append(nozzle_col)
            for waveform_col in waveform_cols:
                nozzle_row_data.append(wf_data[waveform_col])
            #print(nozzle_row_data)
            df.loc[start_index + 2 + ((nozzle_row*nozzles_per_row)+nozzle_col)] = nozzle_row_data
    

In [98]:
serial_num_str = "2D5FV32H"
col_cnt=22
rows_per_head = 4
#nozzles_per_row = 214
nozzles_per_row = 6
lib_cnt = 3
wf_data = WF_var_values

df1 = initialize_waveform_df(rows_per_head, nozzles_per_row, lib_cnt, col_cnt)
add_serial_data(df1, serial_num_str)

# for lib_num in [2,1,3]:
#     add_nozzle_data(df1, lib_num, rows_per_head, nozzles_per_row, wf_data)

lib2_vals = [20,10,-25,15,-6,3.6,2.4,1,0.4,0.6,1,0.1,0.5,0.2,0.5,0.3,1,1,0.5]
lib1_vals = [0,20,0,0,0,0,0,0,300,0,0,0,0,0,0,0,0,0,0]
lib3_vals = [20,-15,0,0,0,0,0,0,300,0,0,0,0,0,0,0,0,0,0]

waveform_cols = ['BI', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'WT', 'CA', 'HT', 'CB', 'PA', 'CC', 'PB', 'CD', 'PC', 'CE', 'PD', 'CF']
lib2_dict = {}
for val, waveform_col in zip(lib2_vals, waveform_cols):
    lib2_dict[waveform_col] = val
print(lib2_dict)
add_nozzle_data(df1, 0, 2, rows_per_head, nozzles_per_row, lib2_dict)

lib1_dict = {}
for val, waveform_col in zip(lib1_vals, waveform_cols):
    lib1_dict[waveform_col] = val
print(lib1_dict)
add_nozzle_data(df1, 1, 1, rows_per_head, nozzles_per_row, lib1_dict)

lib3_dict = {}
for val, waveform_col in zip(lib3_vals, waveform_cols):
    lib3_dict[waveform_col] = val
print(lib3_dict)
add_nozzle_data(df1, 2, 3, rows_per_head, nozzles_per_row, lib3_dict)

df1

{'BI': 20, 'AA': 10, 'AB': -25, 'AC': 15, 'AD': -6, 'AE': 3.6, 'AF': 2.4, 'WT': 1, 'CA': 0.4, 'HT': 0.6, 'CB': 1, 'PA': 0.1, 'CC': 0.5, 'PB': 0.2, 'CD': 0.5, 'PC': 0.3, 'CE': 1, 'PD': 1, 'CF': 0.5}
{'BI': 0, 'AA': 20, 'AB': 0, 'AC': 0, 'AD': 0, 'AE': 0, 'AF': 0, 'WT': 0, 'CA': 300, 'HT': 0, 'CB': 0, 'PA': 0, 'CC': 0, 'PB': 0, 'CD': 0, 'PC': 0, 'CE': 0, 'PD': 0, 'CF': 0}
{'BI': 20, 'AA': -15, 'AB': 0, 'AC': 0, 'AD': 0, 'AE': 0, 'AF': 0, 'WT': 0, 'CA': 300, 'HT': 0, 'CB': 0, 'PA': 0, 'CC': 0, 'PB': 0, 'CD': 0, 'PC': 0, 'CE': 0, 'PD': 0, 'CF': 0}


0         1       2    3    4    5    6    7    8    9 ...    12   13  \
0   SERI  2D5FV32H                                            ...              
1    LIB         2                                            ...              
2      #       row  column   BI   AA   AB   AC   AD   AE   AF ...    HT   CB   
3    NOZ         0       0   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
4    NOZ         0       1   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
5    NOZ         0       2   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
6    NOZ         0       3   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
7    NOZ         0       4   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
8    NOZ         0       5   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
9    NOZ         1       0   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
10   NOZ         1       1   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
11   NOZ         1       2   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
12   NOZ         1       3   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
13   NOZ         1       4   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
14   NOZ         1       5   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
15   NOZ         2       0   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
16   NOZ         2       1   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
17   NOZ         2       2   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
18   NOZ         2       3   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
19   NOZ         2       4   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
20   NOZ         2       5   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
21   NOZ         3       0   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
22   NOZ         3       1   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
23   NOZ         3       2   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
24   NOZ         3       3   20   10  -25   15   -6  3.6  2.4 ...   0.6    1   
25   LIB         1                                            ...              
26     #       row  column   BI   AA   AB   AC   AD   AE   AF ...    HT   CB   
27   NOZ         0       0    0   20    0    0    0    0    0 ...     0    0   
28   NOZ         0       1    0   20    0    0    0    0    0 ...     0    0   
29   NOZ         0       2    0   20    0    0    0    0    0 ...     0    0   
..   ...       ...     ...  ...  ...  ...  ...  ...  ...  ... ...   ...  ...   
49   LIB         3                                            ...              
50     #       row  column   BI   AA   AB   AC   AD   AE   AF ...    HT   CB   
51   NOZ         0       0   20  -15    0    0    0    0    0 ...     0    0   
52   NOZ         0       1   20  -15    0    0    0    0    0 ...     0    0   
53   NOZ         0       2   20  -15    0    0    0    0    0 ...     0    0   
54   NOZ         0       3   20  -15    0    0    0    0    0 ...     0    0   
55   NOZ         0       4   20  -15    0    0    0    0    0 ...     0    0   
56   NOZ         0       5   20  -15    0    0    0    0    0 ...     0    0   
57   NOZ         1       0   20  -15    0    0    0    0    0 ...     0    0   
58   NOZ         1       1   20  -15    0    0    0    0    0 ...     0    0   
59   NOZ         1       2   20  -15    0    0    0    0    0 ...     0    0   
60   NOZ         1       3   20  -15    0    0    0    0    0 ...     0    0   
61   NOZ         1       4   20  -15    0    0    0    0    0 ...     0    0   
62   NOZ         1       5   20  -15    0    0    0    0    0 ...     0    0   
63   NOZ         2       0   20  -15    0    0    0    0    0 ...     0    0   
64   NOZ         2       1   20  -15    0    0    0    0    0 ...     0    0   
65   NOZ         2       2   20  -15    0    0    0    0    0 ...     0    0   
66   NOZ         2       3   20  -15    0    0    0    0    0 ...     0    0   
67   NO

In [50]:
header_fields = [str(x) for x in range(num_cols)]
df = pd.DataFrame(columns=header_fields)
fields = ["SERI", "2D5FV32H"]
df.loc[0] = fields + ["" for x in range(len(df.columns)-len(fields))]

df.head()

AttributeError: 'DataFrame' object has no attribute 'row_cnt'

In [18]:
col_fields = ['row', 'column', 'BI', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'WT', 'CA', 'HT', 'CB', 'PA', 'CC', 'PB', 'CD', 'PC', 'CE', 'PD', 'CF']
len(col_fields)

21

In [67]:
v = list(WF_var_values.values())
print(v)
[5,6,7] + [1,2,3]

[30, 5, -10, 10, -6, -5, 2.4, 0.4, 0.6, 1, 0.3, 1, 1.2, 0.5, 1, 0.8, 0.7, 1, 1]


[5, 6, 7, 1, 2, 3]

In [61]:
WF_var_values

{'BI': 30,
 'AA': 5,
 'AB': -10,
 'AC': 10,
 'AD': -6,
 'AE': -5,
 'AF': 2.4,
 'CA': 0.4,
 'HT': 0.6,
 'CB': 1,
 'PA': 0.3,
 'CD': 1,
 'PC': 1.2,
 'CF': 0.5,
 'WT': 1,
 'CC': 0.8,
 'PB': 0.7,
 'CE': 1,
 'PD': 1}